In [1]:
# Capture the version of the sklearn library
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [2]:
# Install packages needed
import pickle
import pandas as pd
import os

In [3]:
# Load the model present in the homework directory
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    """ Read data given a pasth"""
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
# Read the yellow taxi data from February 2022
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.0,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25,12.433333
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.0,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25,17.550000
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.5,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25,23.650000
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.0,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00,20.083333
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.5,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25,26.316667


In [7]:
# Transform using the dictionary vectorizer
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)

# Get the predictions of the model and print the standard deviation
y_pred = model.predict(X_val)
print("Standard deviation:", round(y_pred.std(), 2))

# QUESTION 1

Standard deviation: 5.28


In [22]:
d = {'year': 2022, 'month': 2}
**d

SyntaxError: invalid syntax (692680287.py, line 2)

SyntaxError: invalid syntax (4231487288.py, line 1)

In [8]:
# Create the ride_id column
year = 2022
month = 2

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [9]:
# Create the df_result that contains two columns; "ride_id" and "pred"
df_result = pd.concat([pd.Series(df["ride_id"]).reset_index(drop = True), pd.Series(y_pred).reset_index(drop = True)], axis=1).rename({0:" pred"}, axis=1)
df_result.head()

,ride_id,pred
0,2022/02_0,18.527783
1,2022/02_1,23.065782
2,2022/02_2,33.686359
3,2022/02_3,23.757436
4,2022/02_4,21.492904


In [10]:
# Define the name of the output file
output_file = os.getcwd() + '/output_file.parquet'
output_file

'/Users/atawua/Desktop/Data Science/GitHub/mlops-zoomcamp/cohorts/2023/04-deployment/homework/output_file.parquet'

In [13]:
# Save the df_result as a parquet file
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

# Get the size of the parquet file
os.path.getsize('./output_file.parquet')

# QUESTION 2

59994818

In [16]:
# Turn the starter jupyter notebook into a .py script
!jupyter nbconvert --to script starter.ipynb

# QUESTION 3

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 2231 bytes to starter.py


In [ ]:
# !pipenv install sckikit-learn==1.2.2 flask --python=3.9
# !pipenv shell